## Teil 2: Analyse der im Datenset enthaltenen Personen

Im folgenden sollen die verschiedenen im Datenset erwähnten Personen aus diesem extrahiert und analysiert werden:

### Importieren der benötigten Bibliotheken

In [ ]:
import pandas as pd
from lxml import etree
import unicodedata
import plotly.express as px
import numpy as np

### Exkurs anhand eines exemplarischen Datensatzes: 

Beispiel-XML

<div>
<img src="example.jpg">
    
</div>

In [ ]:
# Laden der MARC-xml-Datei in ElementTree und umwandeln in eine Liste an Datensätzen: 
tree = etree.parse('example.xml')
root = tree.getroot()   
records = list(root)
print("Gefundene Records:", len(records))

In [ ]:
# Funktion zum Extrahieren von Datensätzen
def parse_record(record):
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    
    def extract_text(xpath_query):
        fields = record.xpath(xpath_query, namespaces=ns)
        if fields:
            return "; ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in fields if field.text)
        return "unknown"

    idn = extract_text("marc:controlfield[@tag='001']")
    titel = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    author = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    author_rela = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='e']")
    add_author = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='a']")
    add_author_rela = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='e']")
    jahr = extract_text("marc:datafield[@tag='264']/marc:subfield[@code='c']")

    return {
        "idn": idn,
        "titel": titel,
        "author": author,
        "author_rela": author_rela,
        "additional_author": add_author,
        "added_rela": add_author_rela,
        "jahr": jahr        
    }

In [ ]:
# Übergabe der einzelnen Datensätze an die Funktion "parse_record":
result = [parse_record(record) for record in records]
df = pd.DataFrame(result)
df

Die einzelnen Einträge unter additional_author und added_rela sind jeweils durch ein Semikolon getrennt - da es im Eintrag zu Franz Siegfried Müller allerdings zwei Einträge der Relation (zwei subfields $e) für beide Funktionen als Verfasser sowie als Herausgeber gab, sind die Einträge hier nicht mehr eindeutig zuordnenbar (2 Einträge zu additional_authors vs. 3 added_rela-Einträge). 

Für eine genauere Analyse der einzelnen Personen inklusive der Beziehungen zum Werk ist es also nötig, die Funktion anzupassen und die einzelnen Unterfelder in Abhängigkeit ihrer "datafields" zu extrahieren und die Abhängigkeiten entsprechend abzubilden. Dazu wird die Funktion von eben etwas verändert und erweitert: 

In [ ]:
# Erweiterte Funktion zum Extrahieren der Inhalte: 
def parse_record_advanced(record):
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}

    def extract_text(xpath_query):
        element = xpath_query.split("/")
        parentpath = xpath_query.split("/")[0]
        if len(element) == 1:
            fields = record.xpath(xpath_query, namespaces=ns)
            if fields:
                return "; ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in fields if field.text)
            return "unknown"
        else:
            childpath = xpath_query.split("/")[1]
            parentfields = record.xpath(parentpath, namespaces=ns)
            result = []
            
            for parent in parentfields:
                subfields = parent.xpath(childpath, namespaces=ns)
                if subfields:
                    text = " / ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in subfields if field.text)
                else:
                    text = ""
                result.append(text)
    
            if len(result) == 0:
                return None
            elif len(result) == 1:
                return result[0]
            else:
                return result
                

    # Extract fields
    idn = extract_text("marc:controlfield[@tag='001']")
    titel = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    author = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    author_rela = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='e']")
    add_author = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='a']")
    add_author_rela = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='e']")
    jahr = extract_text("marc:datafield[@tag='264']/marc:subfield[@code='c']")
    
    return {
        "idn": idn,
        "titel": titel,
        "author": author,
        "author_rela": author_rela,
        "additional_author": add_author,
        "added_rela": add_author_rela,
        "jahr": jahr        
    }


In [ ]:
result_new = [parse_record_advanced(record) for record in records]
df_new = pd.DataFrame(result_new)
df_new

### Anwenden der neuen Funktion auf den gesamten Datensatz: 

In [ ]:
# Laden der MARC-xml-Datei in ElementTree und umwandeln in eine Liste an Datensätzen: 
tree2 = etree.parse('../../dataset_tutorial_eco.xml')
root2 = tree2.getroot()                
records2 = list(root2)
print("Gefundene Records:", len(records2))

In [ ]:
result_complete = [parse_record_advanced(record) for record in records2]
df_new = pd.DataFrame(result_complete)
df_new

### Unwandeln des Dataframes für die weitere Analyse

Das aktuelle Dataframe verfügt nun über jeweils 2 Spalten, in denen Personen gelistet werden, sowie zwei zugehörige Spalten, die deren Relation zum Werk beschreiben. Um im weiteren Verlauf alle Personen gleichermaßen listen und analyiseren zu können, muss das Dataframe nun noch etwas angepasst werden. 

Ziel ist es, nur noch eine Spalte "person" mit einer zugehörigen Spalte "person_rela" zu erzeugen und die Daten aus author, author_rela, additional_author und added_rela entsprechend in diese Spalten zu überführen. Außerdem soll die Beziehung der jeweiligen Person zum Werk erhalten bleiben. Das neue Dataframe sollte also in etwas folgendermaßen aussehen: 

| idn        | titel	                                         | person                         | person_rela	          | jahr | 
|------------|---------------------------------------------------|--------------------------------|-----------------------|------| 
| 1043718966 | A cross-cultural study of motivational factors... | Juarez Hernandez, Maria Isabel | Verfasser             | 2010 |
| 1043718966 | A cross-cultural study of motivational factors... |	Spiller, Achim 	              | Akademischer Betreuer | 2010 |
| 1043718966 | A cross-cultural study of motivational factors... |	Theuvsen, Ludwig 	          | Akademischer Betreuer | 2010 |
| 1019902175 | A forecast evaluation of PCA-based adaptive fo... | Blaskowitz, Oliver Jim	      | Verfasser             | 2010 |
| 1012057232 | A new perspective on social learning              | March, Christoph               | Verfasser             | None |
| usw. | | | | |

Um dies zu erreichen sind mehrere Schritte nötig: Zunächst werden die Listen, die sich in `additional_author` und `added_rela` befinden mit Hilfe der .explode()-Funktion von Pandas "entpackt". Dies bewirkt, dass eine Spalte, die Listen oder ähnliche Strukturen enthält so dupliziert wird, dass jede Listeneinheit in eine eigene Zeile aufgeteilt wird, während die anderen Spalten unverändert bleiben. Gleichzeitig werden die Spalten `author` und `author_rela` für die gewünschte neue Struktur umbenannt:



In [ ]:
df_explode = df_new.explode(['additional_author', 'added_rela'])
df_persons = df_explode.rename(columns={'author': 'person', 'author_rela': 'person_rela'}).copy()
df_persons

Nun wird ein neues Dataframe erstellt, welches folgendermaßen befüllt wird: 

- Kopieren jeder Zeile in das neue Dataframe 
- Prüfung, ob ein Eintrag unter `additional_author` vorhanden ist. Wenn ja: 
    - Erneutes Kopieren der Zeile
    - Überschreiben der Einträge in `person` und `person_rela` mit den Werten aus `additional_author` und `added_rela` in der neuen Zeile

In [ ]:
# Erstellen des neuen Dataframes: 
new_rows = []

for index, row in df_persons.iterrows():
    new_rows.append(row) # Kopieren der Zeilen

    if pd.notna(row['additional_author']):
        new_row = row.copy() 
        new_row['person'] = row['additional_author'] 
        new_row['person_rela'] = row['added_rela']  
        new_rows.append(new_row)  

new_df = pd.DataFrame(new_rows)
new_df.head()


Entfernen der Spalten "additional_author" und "added_rela" mit anschließender Löschung doppelter Zeilen sowie von Zeilen, die keinen Eintrag unter "person" haben (da sie ursprünglich bspw. nicht über einen 100er Eintrag, sondern nur über einen 700er Eintrag verfügten): 

In [ ]:
new_df = new_df.drop(columns=['additional_author', 'added_rela'])
new_df.drop_duplicates(inplace=True)
new_df.dropna(subset=['person'], inplace=True)
new_df

In [ ]:
# Optional: Speichern des Dataframes als CSV
new_df.to_csv("personen.csv", encoding="utf-8")

### Analyse:

Gruppieren und Zählen der einzelnen Personen: 

In [ ]:
# Häufigkeit der Autoren zählen
person_counts = new_df['person'].value_counts().head(25).reset_index()
person_counts

In [ ]:
# Balkendiagramm erstellen
fig1 = px.bar(person_counts, x='person', y='count', height=550,
             title='Häufigste Personen im Datenset', color='count', color_continuous_scale='Viridis')
fig1.show()

Suche im Dataframe nach einer bestimmten Person: 

In [ ]:
new_df.loc[new_df['person'] == 'Härdle, Wolfgang']

Häufigkeiten der verschiedenen Rollen:

In [ ]:
# Häufigkeit von "person_rela"
rela_counts = new_df['person_rela'].value_counts().head(25).reset_index()

# Kreisdiagramm erstellen
fig2 = px.pie(rela_counts, values='count', names='person_rela', title='Verteilung der Rollen (`person_rela`) im Datenset', 
              color_discrete_sequence=px.colors.sequential.RdBu, height=500)
fig2.show()